# Using pantherdb to get the orthologs from ref proteom

In [1]:
import pandas as pd
import Bio
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [24]:
experiment_name = "MOUSE2HUMAN"
back_path = "../../"
orth_file = "MOUSE_HUMAN_ref.tab"
exp_protein_file = "MOUSE_invadosomes.txt"
FROM="MOUSE"
TO="HUMAN"

In [12]:
with open(back_path+'05_Output/'+experiment_name+"/"+experiment_name+'_matches2refproteom.pickle', 'rb') as f:
    uniprot2seq = pickle.load(f)

# Retrieving the orthologs

In [13]:
orthologs_table = pd.read_csv(back_path+'01_Reference/'+experiment_name+"/"+orth_file,sep='\t',header=None)

orthologs_table.head()

,0,1,2,3,4
0,HUMAN|HGNC=11477|UniProtKB=Q15528,MOUSE|MGI=MGI=98446|UniProtKB=Q62276,LDO,Euarchontoglires,PTHR12434
1,HUMAN|HGNC=28143|UniProtKB=Q53S58,MOUSE|MGI=MGI=1913593|UniProtKB=Q8BPE4,LDO,Euarchontoglires,PTHR21824
2,HUMAN|HGNC=3042|UniProtKB=Q9UKA8,MOUSE|MGI=MGI=1858220|UniProtKB=Q9JKK0,LDO,Euarchontoglires,PTHR10300
3,HUMAN|HGNC=3040|UniProtKB=P53805,MOUSE|MGI=MGI=1890564|UniProtKB=Q9JHG6,LDO,Euarchontoglires,PTHR10300
4,HUMAN|HGNC=3041|UniProtKB=Q14206,MOUSE|MGI=MGI=1858219|UniProtKB=Q9JHG2,LDO,Euarchontoglires,PTHR10300


In [14]:
d = {}
for i in orthologs_table.index:
    if TO in orthologs_table[0][i]:
        for el in orthologs_table[0][i].split("|"):
            if "UniProtKB" in el:
                to = el.split("UniProtKB=")[1]
        for el in orthologs_table[1][i].split("|"):
            if "UniProtKB" in el:
                var_from = el.split("UniProtKB=")[1]
    else:
        for el in orthologs_table[1][i].split("|"):
            if "UniProtKB" in el:
                to = el.split("UniProtKB=")[1]
        for el in orthologs_table[0][i].split("|"):
            if "UniProtKB" in el:
                var_from = el.split("UniProtKB=")[1]
    store = (to,orthologs_table[2][i])
    if var_from in d:
        d[var_from].append(store)
    else:
        d[var_from] = [store]

In [15]:
len(d)

19050

In [16]:
d['Q62276']

[('Q15528', 'LDO')]

# Matching from old uniprot id to orthologs 

In [17]:
orthologs_found = {}
orthologs_not_found = {}

for key, values in uniprot2seq.items():
    found = False
    for value in values:
        from_ref_id = value.name.split("|")[1]
        if from_ref_id not in d:
            continue
        else:
            to_ref_id = ""
            b=True
            for el in d[from_ref_id]:
                if el[1] == 'LDO':
                    to_ref_id = el[0]
                    b=False
                    break
                else:
                    to_ref_id = el[0]
            if b :
                print(key,from_ref_id,to_ref_id)
            orthologs_found[key] = to_ref_id
            found = True
            break
            
    if not found :
        orthologs_not_found[key] = value

Q91VM5 Q91VM5 P38159
O55137 O55137 Q86TX2
P60766 P60766 P60953


In [18]:
len(orthologs_found)

306

In [19]:
len(orthologs_not_found)

6

In [10]:
uniprot2seq["J9NXT1"]

[SeqRecord(seq=Seq('MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFS...EEY'), id='tr|J9NXT1|J9NXT1_CANLF', name='tr|J9NXT1|J9NXT1_CANLF', description='tr|J9NXT1|J9NXT1_CANLF Tubulin alpha chain OS=Canis lupus familiaris OX=9615 GN=TUBA3D PE=3 SV=1', dbxrefs=[]),
 SeqRecord(seq=Seq('MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFS...EEY'), id='tr|J9NXT1|J9NXT1_CANLF', name='tr|J9NXT1|J9NXT1_CANLF', description='tr|J9NXT1|J9NXT1_CANLF Tubulin alpha chain OS=Canis lupus familiaris OX=9615 GN=TUBA3D PE=3 SV=1', dbxrefs=[]),
 SeqRecord(seq=Seq('MRECISVHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFS...EEY'), id='tr|F1PE21|F1PE21_CANLF', name='tr|F1PE21|F1PE21_CANLF', description='tr|F1PE21|F1PE21_CANLF Tubulin alpha chain OS=Canis lupus familiaris OX=9615 GN=LOC477570 PE=3 SV=2', dbxrefs=[]),
 SeqRecord(seq=Seq('MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFS...EEY'), id='tr|F1PM53|F1PM53_CANLF', name='tr|F1PM53|F1PM53_CANLF', description='tr|F1PM53|F1PM53_CANLF Tubulin al

In [11]:
orthologs_found["J9NXT1"]

'P0DPH8'

# lets check how many lines we get from the filtered set of proteins

In [26]:
filtered_prots = pd.read_csv(back_path+"00_InputData/"+experiment_name+"/"+exp_protein_file, sep="\t",header=None)
filtered_prots

,0
0,P61514
1,Q4FK74
2,Q01149
3,O88544
4,P97310
...,...
307,A0A140T8M7
308,G3UX26
309,Q922B2
310,Q8VDD5


In [28]:
list_of_found_indices = []
list_of_not_found_indices = []
for i in filtered_prots.index:
    s = filtered_prots[0][i]
    l = s.split(";")
    for el in l:
        if el in orthologs_found:
            list_of_found_indices.append(i)
            break
    if list_of_found_indices[-1] != i:
        list_of_not_found_indices.append(i)

In [29]:
len(list_of_found_indices)

306

In [30]:
len(list_of_not_found_indices)

6

In [31]:
len(list_of_not_found_indices)+len(list_of_found_indices)

312

In [32]:
print(list_of_not_found_indices)

[7, 81, 91, 94, 102, 199]


In [33]:
found_orthologs_table = pd.DataFrame([[key,val] for key, val in orthologs_found.items()],columns= [FROM+"_UniProtKB",TO+"_UniprotKB"])
found_orthologs_table

,MOUSE_UniProtKB,HUMAN_UniprotKB
0,P61514,P61513
1,Q4FK74,P30049
2,Q01149,P08123
3,O88544,Q9BT78
4,P97310,P49736
...,...,...
301,F7CVJ5,Q09666
302,E9PYL9,P62913
303,D3YW87,Q14315
304,D3YVN7,P49411


In [34]:
not_found_orthologs_table = pd.DataFrame([[key,np.nan] for key, val in orthologs_not_found.items()],columns= [FROM+"_UniProtKB",TO+"_UniprotKB"])
not_found_orthologs_table

,MOUSE_UniProtKB,HUMAN_UniprotKB
0,A0A140LJ59,NaN
1,D3Z1Z8,NaN
2,A0A0G2JDW7,NaN
3,A0A0A6YW67,NaN
4,Q9D1E6,NaN
5,A0A140T8K6,NaN


In [35]:
matches = pd.concat([found_orthologs_table,not_found_orthologs_table])
matches

,MOUSE_UniProtKB,HUMAN_UniprotKB
0,P61514,P61513
1,Q4FK74,P30049
2,Q01149,P08123
3,O88544,Q9BT78
4,P97310,P49736
...,...,...
1,D3Z1Z8,NaN
2,A0A0G2JDW7,NaN
3,A0A0A6YW67,NaN
4,Q9D1E6,NaN


In [36]:
matches.to_csv(back_path+"05_Output/"+experiment_name+"/"+experiment_name+"_matches_uniprotKB.tsv",sep="\t",index=None)